In [1]:
import glob
import numpy as np
import pandas as pd
import json
import datetime
import pymongo
import sys
from pymongo import MongoClient

In [2]:
def export_content():
    date = datetime.datetime.now()
    date = str(date)[:-16]
    client = MongoClient('mongodb://localhost:27017/')
    db = client['GJNHS_database']
    collection = db['patients']
    cursor = collection.find()
    docs = list(cursor)
    df = pd.DataFrame(docs)
    if df.empty == False:
        df = df.drop(df.columns[0],axis=1)
        df.to_csv('./exported_csv.csv',index=False)
        df.to_csv('./exporthistory/exported_csv_'+date+'.csv',index=False)
        print('Content imported from MongoDB as CSV')
        print('Content ready to update')
        return(df)
    else:
        print('MongoDB database is empty. No data exported.')
        return(df)
    
def update_content(df_mongo,df_new):
    if df_mongo.empty==False and df_new.empty==False:
        df = df_new[df_new['CHI'].isin(df_mongo['CHI']) == False]
        if df.empty==False:
            print('CSV updated with new patient CHI number, ready To import on mongoDB')
            return(df)
        else:
            print('No new CHI number detected, MongoDB up to date')
            return(df)
    else:
        return(print('Dataframe was empty'))

def import_new_content(df,df_new):
    client = MongoClient('mongodb://localhost:27017/')
    db = client['GJNHS_database']
    collection = db['patients']
    data = df
    try:
        if data.empty==False:
            collection.insert_many(data.to_dict('records'))
            return(print('Updated CSV imported on MongoDB'))
    except:
        return(print('No new patient found for update'))
    finally:
        try:
            if df_new.empty==False:
                collection.insert_many(df_new.to_dict('records'))
                return(print('CSV with new patients imported to MongoDB'))
        except:
            return(print('No new patient found, CSV not imported to Mongodb'))

In [3]:
df_new = pd.read_csv('./extracted_data.csv',parse_dates=['Clinic Date'])

In [4]:
df_mongo = export_content()
df = update_content(df_mongo,df_new)
import_new_content(df,df_new)

Content imported from MongoDB as CSV
Content ready to update
No new CHI number detected, MongoDB up to date
No new patient found, CSV not imported to Mongodb
